In [1]:
# create spark sesh
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, monotonically_increasing_id, mean, lit
from pyspark.sql.types import IntegerType, StringType

spark = SparkSession.builder.master("local[2]") \
    .appName("Card Dataset Analysis") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


In [3]:
file_path = "sample_data/cards (2).json"
data = spark.read.json(file_path)
data.printSchema()
data.show(5)

root
 |-- artist: string (nullable = true)
 |-- attack: long (nullable = true)
 |-- cardClass: string (nullable = true)
 |-- classes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- collectible: boolean (nullable = true)
 |-- collectionText: string (nullable = true)
 |-- cost: long (nullable = true)
 |-- dbfId: long (nullable = true)
 |-- durability: long (nullable = true)
 |-- elite: boolean (nullable = true)
 |-- entourage: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- faction: string (nullable = true)
 |-- flavor: string (nullable = true)
 |-- health: long (nullable = true)
 |-- hideStats: boolean (nullable = true)
 |-- howToEarn: string (nullable = true)
 |-- howToEarnGolden: string (nullable = true)
 |-- id: string (nullable = true)
 |-- mechanics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- multiClassGroup: string (nullable = true)
 |-- name: string (nullable = true)
 |-- overload: long (

In [4]:
# remove samples without 'artist', 'attack', or 'health' values
data_cleaned = data.filter(col("artist").isNotNull() & col("attack").isNotNull() & col("health").isNotNull()).cache()
data_cleaned.show(5)

+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+-----------------+--------+----------------+-----------+----+---------+--------------+-------+-----------+------------------+--------------------+------+
|       artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|entourage| faction|              flavor|health|hideStats|           howToEarn|     howToEarnGolden|     id|  mechanics|multiClassGroup|             name|overload|playRequirements|playerClass|race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|  type|
+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+--

In [5]:
# num of non-null rows and unique values
for column_name in data_cleaned.columns:
    non_null_count = data_cleaned.filter(col(column_name).isNotNull()).count()
    unique_count = data_cleaned.select(column_name).distinct().count()
    print(f"Column: {column_name} | Non-Null Rows: {non_null_count} | Unique Values: {unique_count}")

Column: artist | Non-Null Rows: 829 | Unique Values: 234
Column: attack | Non-Null Rows: 829 | Unique Values: 31
Column: cardClass | Non-Null Rows: 829 | Unique Values: 10
Column: classes | Non-Null Rows: 9 | Unique Values: 4
Column: collectible | Non-Null Rows: 731 | Unique Values: 2
Column: collectionText | Non-Null Rows: 5 | Unique Values: 6
Column: cost | Non-Null Rows: 829 | Unique Values: 14
Column: dbfId | Non-Null Rows: 829 | Unique Values: 829
Column: durability | Non-Null Rows: 0 | Unique Values: 1
Column: elite | Non-Null Rows: 141 | Unique Values: 2
Column: entourage | Non-Null Rows: 6 | Unique Values: 7
Column: faction | Non-Null Rows: 60 | Unique Values: 3
Column: flavor | Non-Null Rows: 731 | Unique Values: 732
Column: health | Non-Null Rows: 829 | Unique Values: 30
Column: hideStats | Non-Null Rows: 2 | Unique Values: 2
Column: howToEarn | Non-Null Rows: 136 | Unique Values: 27
Column: howToEarnGolden | Non-Null Rows: 181 | Unique Values: 77
Column: id | Non-Null Rows: 

In [6]:
rarity_mapping = {
    'FREE': 1, 'COMMON': 2, 'RARE': 3,
    'EPIC': 4, 'LEGENDARY': 5, None: 0
}
map_rarity = udf(lambda r: rarity_mapping.get(r, 0), IntegerType())  # map rarity to int values using a UDF
data_with_rarity = data_cleaned.withColumn("rarity_int", map_rarity(col("rarity")))
data_with_rarity.show(5)

+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+-----------------+--------+----------------+-----------+----+---------+--------------+-------+-----------+------------------+--------------------+------+----------+
|       artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|entourage| faction|              flavor|health|hideStats|           howToEarn|     howToEarnGolden|     id|  mechanics|multiClassGroup|             name|overload|playRequirements|playerClass|race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|  type|rarity_int|
+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+--------

In [7]:
# calc avg rarity for each card class
average_rarity = data_with_rarity.groupBy("cardClass").agg(mean("rarity_int").alias("average_rarity"))
average_rarity.show()

+---------+------------------+
|cardClass|    average_rarity|
+---------+------------------+
|   HUNTER|2.4761904761904763|
|   SHAMAN|               3.0|
|  PALADIN| 2.966666666666667|
|     MAGE|2.9393939393939394|
|   PRIEST| 2.909090909090909|
|  WARLOCK|2.4347826086956523|
|  NEUTRAL|2.6706586826347305|
|  WARRIOR| 2.735294117647059|
|    DRUID| 2.789473684210526|
|    ROGUE|3.0526315789473686|
+---------+------------------+



In [8]:
# def a UDF -> number of attacks needed for a card to kill itself
@udf(returnType=IntegerType())
def calculate_self_kills(health, attack):
    if attack == 0:
        return -1  # cannot attack
    return health // attack + (1 if health % attack != 0 else 0)

data_with_attacks = data_with_rarity.withColumn("self_kill_attacks", calculate_self_kills(col("health"), col("attack")))
data_with_attacks.select("name", "health", "attack", "self_kill_attacks").show(5)

+-----------------+------+------+-----------------+
|             name|health|attack|self_kill_attacks|
+-----------------+------+------+-----------------+
|          Icehowl|    10|    10|                1|
|     Wailing Soul|     5|     3|                2|
|Sunfury Protector|     3|     2|                2|
|     Dark Peddler|     2|     2|                1|
| Genzo, the Shark|     4|     5|                1|
+-----------------+------+------+-----------------+
only showing top 5 rows



In [9]:
data_with_ids = data_with_attacks.withColumn("unique_id", monotonically_increasing_id())
data_with_ids.show(5)

+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--------------------+--------------------+-------+-----------+---------------+-----------------+--------+----------------+-----------+----+---------+--------------+-------+-----------+------------------+--------------------+------+----------+-----------------+---------+
|       artist|attack|cardClass|classes|collectible|collectionText|cost|dbfId|durability|elite|entourage| faction|              flavor|health|hideStats|           howToEarn|     howToEarnGolden|     id|  mechanics|multiClassGroup|             name|overload|playRequirements|playerClass|race|   rarity|referencedTags|    set|spellDamage|targetingArrowText|                text|  type|rarity_int|self_kill_attacks|unique_id|
+-------------+------+---------+-------+-----------+--------------+----+-----+----------+-----+---------+--------+--------------------+------+---------+--

In [10]:
# simulate pairing
data_paired = data_with_ids.alias("left").join(
    data_with_ids.alias("right"),
    on=data_with_ids["unique_id"] != data_with_ids["unique_id"], how="inner"
).select(
    col("left.name").alias("left_name"), col("left.health").alias("left_health"), col("left.attack").alias("left_attack"),
    col("right.name").alias("right_name"), col("right.health").alias("right_health"), col("right.attack").alias("right_attack")
)

data_paired.show(5)

+---------+-----------+-----------+-----------------+------------+------------+
|left_name|left_health|left_attack|       right_name|right_health|right_attack|
+---------+-----------+-----------+-----------------+------------+------------+
|  Icehowl|         10|         10|     Wailing Soul|           5|           3|
|  Icehowl|         10|         10|Sunfury Protector|           3|           2|
|  Icehowl|         10|         10|     Dark Peddler|           2|           2|
|  Icehowl|         10|         10| Genzo, the Shark|           4|           5|
|  Icehowl|         10|         10| Reliquary Seeker|           1|           1|
+---------+-----------+-----------+-----------------+------------+------------+
only showing top 5 rows



In [11]:
@udf(returnType=StringType())
def determine_winner(left_health, left_attack, right_health, right_attack):
    while left_health > 0 and right_health > 0:
        right_health -= left_attack
        if right_health <= 0:
            return "Left"
        left_health -= right_attack
    return "Right"

data_battles = data_paired.withColumn("winner", determine_winner(
    col("left_health"), col("left_attack"), col("right_health"), col("right_attack")))
data_battles.select("left_name", "right_name", "winner").show(10)

+---------+-------------------+------+
|left_name|         right_name|winner|
+---------+-------------------+------+
|  Icehowl|       Wailing Soul|  Left|
|  Icehowl|  Sunfury Protector|  Left|
|  Icehowl|       Dark Peddler|  Left|
|  Icehowl|   Genzo, the Shark|  Left|
|  Icehowl|   Reliquary Seeker|  Left|
|  Icehowl|Injured Blademaster|  Left|
|  Icehowl|   Hemet Nesingwary|  Left|
|  Icehowl|         Core Hound|  Left|
|  Icehowl|   Trogg Beastrager|  Left|
|  Icehowl| Captured Jormungar|  Left|
+---------+-------------------+------+
only showing top 10 rows

